In [2]:
from data.data_loader import DroneGraphDataset

ModuleNotFoundError: No module named 'data'

In [ ]:
dataset = DroneGraphDataset(
    trajectory_csv='trajectories.csv',
    relationship_csv='relationships.csv',
    lookback=50,
    device='cuda'
)

sample = dataset[0]
print(sample['context_window'].shape)    # [50, num_drones, 4]
print(sample['current_features'].shape)  # [num_drones, 4]
print(sample['relationships'].shape)     # [num_pairs, 2]
print(sample['labels'].shape)            # [num_pairs]
